In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

dis = 'drive/My Drive/data_final/'
dis_p = 'drive/My Drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
data = pd.read_csv(dis + 'predata_daily.csv', index_col = 'client_dk')

In [ ]:
x_train = data.loc[data['is_train'] == 1]
x_test = data.loc[data['is_train'] == 0]

In [ ]:
test_target = pd.read_csv(dis + 'test.csv')

In [ ]:
x_train = x_train.drop(['is_train'], axis = 1)
x_test = x_test.drop(['is_train'], axis = 1)

In [ ]:
x_test = x_test.reset_index()

In [ ]:
x_test = test_target.set_index('client_dk').merge(x_test, how = 'outer', on = 'client_dk').set_index('client_dk')

In [ ]:
x_test = x_test.drop(list(map(str, [27,32,41,45,67,73,81,88])), axis = 1)
target = x_train[list(map(str, [27,32,41,45,67,73,81,88]))]
x_train = x_train.drop(list(map(str, [27,32,41,45,67,73,81,88])), axis = 1)

In [ ]:
x_train

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,small_group_35,small_group_36,small_group_37,small_group_38,small_group_39,...,day_348,day_349,day_350,day_351,day_352,day_353,day_354,day_355,day_356,day_357,day_358,day_359,day_360,day_361,day_362,day_363,day_364,day_365,day_366,day_367,day_368,day_369,day_370,day_371,day_372,day_373,day_374,day_375,day_376,day_377,day_378,day_379,day_380,day_381,day_382,day_383,day_384,day_385,day_386,day_387
client_dk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9.0,0.0,27.0,8.0,89.0,119.0,309.0,22.0,0.0,12.0,0.0,15.0,15.0,14.0,15.0,1.0,0.0,0.0,2.0,0.0,3.0,0.0,13.0,15.0,0.0,1.0,2.0,0.0,2.0,0.0,24.0,4.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
2,27.0,617.0,196.0,46.0,37.0,114.0,191.0,0.0,1.0,4.0,9.0,79.0,2.0,11.0,3.0,2.0,1.0,0.0,9.0,0.0,1.0,1.0,11.0,0.0,0.0,0.0,0.0,10.0,1.0,2.0,2.0,0.0,2.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
3,16.0,0.0,1.0,19.0,28.0,174.0,320.0,15.0,0.0,23.0,14.0,20.0,23.0,37.0,18.0,3.0,0.0,5.0,0.0,1.0,24.0,0.0,42.0,0.0,1.0,2.0,9.0,1.0,2.0,0.0,0.0,11.0,3.0,0.0,2.0,2.0,0.0,7.0,0.0,1.0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,21.0,0.0,320.0,50.0,25.0,308.0,105.0,45.0,0.0,46.0,3.0,74.0,16.0,3.0,41.0,18.0,4.0,1.0,20.0,1.0,8.0,0.0,34.0,45.0,0.0,2.0,2.0,2.0,1.0,0.0,2.0,24.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
7,14.0,0.0,0.0,21.0,0.0,202.0,89.0,1.0,2.0,45.0,11.0,51.0,30.0,0.0,16.0,78.0,3.0,3.0,6.0,1.0,1.0,1.0,35.0,3.0,0.0,5.0,3.0,15.0,1.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,3.0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,85.0,22.0,262.0,31.0,46.0,103.0,68.0,22.0,12.0,4.0,0.0,78.0,1.0,3.0,19.0,64.0,8.0,0.0,39.0,13.0,0.0,6.0,51.0,2.0,0.0,0.0,0.0,8.0,0.0,61.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,3.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
49995,0.0,0.0,3.0,1.0,0.0,42.0,27.0,0.0,0.0,36.0,55.0,187.0,6.0,0.0,34.0,0.0,0.0,1.0,0.0,0.0,16.0,0.0,19.0,0.0,1.0,8.0,1.0,17.0,38.0,4.0,0.0,8.0,7.0,0.0,1.0,3.0,1.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49996,11.0,0.0,0.0,12.0,12.0,170.0,148.0,0.0,0.0,17.0,10.0,361.0,23.0,0.0,11.0,2.0,1.0,5.0,1.0,3.0,7.0,0.0,30.0,3.0,0.0,3.0,2.0,13.0,0.0,0.0,0.0,2.0,4.0,0.0,5.0,2.0,0.0,4.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
results = {}
for i in target.columns:
  tr = target[i]
  print(i, 'learning...')
  model = CatBoostRegressor(iterations=1500,
                            learning_rate=0.005,
                          depth=3)

  model.fit(x_train, tr)

  results[i] = model.predict(x_test)

Streaming output truncated to the last 5000 lines.
1004:	learn: 0.1372353	total: 1m 55s	remaining: 56.8s
1005:	learn: 0.1372346	total: 1m 55s	remaining: 56.7s
1006:	learn: 0.1372330	total: 1m 55s	remaining: 56.6s
1007:	learn: 0.1372326	total: 1m 55s	remaining: 56.5s
1008:	learn: 0.1372320	total: 1m 55s	remaining: 56.3s
1009:	learn: 0.1372306	total: 1m 55s	remaining: 56.2s
1010:	learn: 0.1372295	total: 1m 55s	remaining: 56.1s
1011:	learn: 0.1372289	total: 1m 56s	remaining: 56s
1012:	learn: 0.1372271	total: 1m 56s	remaining: 55.9s
1013:	learn: 0.1372263	total: 1m 56s	remaining: 55.7s
1014:	learn: 0.1372254	total: 1m 56s	remaining: 55.6s
1015:	learn: 0.1372240	total: 1m 56s	remaining: 55.5s
1016:	learn: 0.1372195	total: 1m 56s	remaining: 55.4s
1017:	learn: 0.1372187	total: 1m 56s	remaining: 55.3s
1018:	learn: 0.1372179	total: 1m 56s	remaining: 55.2s
1019:	learn: 0.1372172	total: 1m 56s	remaining: 55s
1020:	learn: 0.1372162	total: 1m 57s	remaining: 54.9s
1021:	learn: 0.1372154	total: 1m 57

In [ ]:
submission = pd.DataFrame(data=np.zeros((25000,8)),columns = target.columns, index = test_target['client_dk'].values)
submission.columns = ['cat_27','cat_32','cat_41','cat_45','cat_67','cat_73','cat_81','cat_88']

In [ ]:
for q in results:
    submission['cat_'+str(q)] = results[q]

In [ ]:
for q in submission.columns:
    submission[q][submission[q] < 0] = 0

In [ ]:
submission

,cat_27,cat_32,cat_41,cat_45,cat_67,cat_73,cat_81,cat_88
12671,0.048977,0.194626,0.154253,0.397901,0.011308,0.019587,0.001720,0.009796
14015,0.042005,0.079049,0.069334,0.473984,0.009306,0.044381,0.001097,0.007717
1500,0.041111,0.057326,0.041689,0.450961,0.009125,0.017420,0.001435,0.008450
4934,0.146688,0.290680,0.084413,0.490657,0.011258,0.049092,0.002009,0.010398
11405,0.045851,0.036373,0.071795,0.071061,0.865508,0.014766,0.001578,0.006839
...,...,...,...,...,...,...,...,...
31376,0.034327,0.618649,0.287724,0.067780,0.010559,0.064498,0.001346,0.007422
14570,0.133173,0.123207,0.180515,0.263401,0.011069,0.035499,0.001300,0.010155
35249,0.039099,0.435914,0.095876,0.258179,0.011252,0.158972,0.010999,0.013650
10269,0.039125,0.063426,0.059518,0.554472,0.011010,0.034532,0.001373,0.006049


In [ ]:
submission.index.name = 'client_dk'
submission.to_csv(dis_p + 'predict_daily1.csv')